In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import LineString, Point, MultiPolygon
import requests
import json

import matplotlib.pyplot as plt

# Step 1: Download and process OSM  data

In [ ]:
def get_osm_road_data(country):

    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = """
    /*
    This shows the roads in the country selected. Try Bahrain: البحرين
    */

    [out:json];
    area[name="<country>"];
     (way["highway"~"motorway|trunk|primary|motorway_link|trunk_link|primary_link|unclassified|tertiary|secondary|track|path|residential|service|secondary_link|tertiary_link"](area);
    );
    out ids geom;

    """

    overpass_query = overpass_query.replace("<country>", country)

    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()
    
    return data

In [ ]:
country = "البحرين" #This is Bahrain
data = get_osm_road_data(country)

The code above works. But we may want to modify it so that I can also get the road names as well.

In [ ]:
def way_to_line(way):
    """
    Takes a way from OSM as an input.
    Returns a shapely line object.
    """
    coords = []
    for i in range(len(way["geometry"])):
        coord = (way.get("geometry")[i]["lon"], way.get("geometry")[i]["lat"])
        coords.append(coord)
    line = LineString(coords)
    return line

In [ ]:
def overpass_query_to_df(data):
    """
    Takes a raw query from OSM as an input.
    Returns a geopandas style dataframe.
    """
    lines = []
    ids = []
    for i in range((len(data.get("elements")))):
        way = data.get("elements")[i]
        road_id = way["id"]
        line = way_to_line(way)
        lines.append(line)
        ids.append(road_id)
    df = pd.DataFrame([lines,ids]).T.rename(columns = {0 : "geometry", 1: "way_id"})
    return df

In [ ]:
roads = overpass_query_to_df(data)
roads = gpd.GeoDataFrame(roads)

# Step 2: Calculate total length of road in cell

In [ ]:
def length_of_road_in_cell(roads_gpd, cell):
    length = 0
    for i in range(len(roads_gpd)):
        k = cell.intersection(roads_gpd["geometry"][i])
        if k:
            length += k.length
    return length

In [ ]:
delta = .01
x_min, y_min, x_max, y_max = roads.total_bounds

In [ ]:
x_array = np.arange(x_min,x_max, delta)
y_array = np.arange(y_min, y_max, delta)

In [ ]:
def calculate_road_length_by_cell(x_range=x_array, y_range=y_array,roads_gpd=roads, delta = delta):
    cells = [] #This is just a temporary addition
    
    sums = []
    lon = []
    lat = []

    for i in x_range:
        for j in y_range:
            cell = (Point(i,j).buffer(delta/2, cap_style=3))
            
            cells.append(cell) #A temporary return just so we can see the grid
            
            
            sums.append(length_of_road_in_cell(roads_gpd, cell))
            lon.append(i)
            lat.append(j)

    df = pd.DataFrame([lon,lat,sums]).T.rename(columns = {0: "lon",1:"lat",2:"road_length"})
    
    return df, cells

In [ ]:
df, cells = calculate_road_length_by_cell()

***

**Now just a little bit of work to check on my work in the above....**

***


In [ ]:
bahrain = gpd.read_file("BHR_adm0.shp")

In [ ]:
bahrain.plot()
roads.plot()

In [ ]:
i=x_array[0]
j=y_array[0]
square = Point(i,j).buffer(delta/2, cap_style=3)

plt.plot(square.exterior.xy[0],square.exterior.xy[1])

In [ ]:
print("lon delta:", square.bounds[0] - square.bounds[2])
print("lat delta:", square.bounds[1] - square.bounds[3])

In [ ]:
square.bounds

In [ ]:
MultiPolygon(cells)

In [ ]:
pd.set_option('display.max_rows', 500)
df